In [1]:

#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


--


#  Setup stuff: Connectivity

In [2]:

from katana import remote
from katana.remote import import_data


my_client = remote.Client()

print(my_client)


In [3]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


--


In [23]:

#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")


--


In [ ]:

#  DELETE ALL DATABASES

for l_database in my_client.databases():
   if (l_database.name != "default"):
      my_client.get_database(name=l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)


In [ ]:

#  CREATE DATABASE

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)


In [24]:

#  CREATE GRAPH
#
my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)


<_Graph my_graph, J2DgAed6fCQqwfqLyQHifg5ragT3vtDF3vzvoJnzNJaC, 0>


In [ ]:

#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


# Create: Vertices/nodes, edges ..

In [ ]:
# import pandas as pd
# 
# print("--")


In [25]:

#  Create Vertices/Nodes ..

df_airports = pd.DataFrame([
       #
   ["MKE" , "Milwaukee"               , "Airport"],
   ["ORD" , "Chicago O-Hare"          , "Airport"],
   ["SJC" , "San Jose"                , "Airport"],
   ["DEN" , "Denver"                  , "Airport"],
       #
   ], columns = ["id", "airport_name", "LABEL"])
      #
print(tabulate(df_airports, headers='keys', tablefmt='psql'))


+----+------+----------------+---------+
|    | id   | airport_name   | LABEL   |
|----+------+----------------+---------|
|  0 | MKE  | Milwaukee      | Airport |
|  1 | ORD  | Chicago O-Hare | Airport |
|  2 | SJC  | San Jose       | Airport |
|  3 | DEN  | Denver         | Airport |
+----+------+----------------+---------+


In [26]:

#  Create the Edge/Relationships ..

df_relates_to_N = pd.DataFrame([
      #
   ["DEN", "ORD", "FLIGHT"],
   ["DEN", "MKE", "FLIGHT"],
   ["DEN", "SJC", "FLIGHT"],
   ["MKE", "DEN", "FLIGHT"],
      #
   ], columns = ["START_ID", "END_ID", "TYPE"])
      #
print(tabulate(df_relates_to_N, headers='keys', tablefmt='psql'))


+----+------------+----------+--------+
|    | START_ID   | END_ID   | TYPE   |
|----+------------+----------+--------|
|  0 | DEN        | ORD      | FLIGHT |
|  1 | DEN        | MKE      | FLIGHT |
|  2 | DEN        | SJC      | FLIGHT |
|  3 | MKE        | DEN      | FLIGHT |
+----+------------+----------+--------+


In [27]:

#  Import the previously created Python PandasDataFrames into KatanaGraph ..


with import_data.DataFrameImporter(my_graph) as df_importer:   
   df_importer.nodes_dataframe(
      df_airports,
      id_column             = "id",
      id_space              = "Airport",  
      label                 = "Airport",  
      )
   df_importer.insert()
 
with import_data.DataFrameImporter(my_graph) as df_importer:   
   df_importer.edges_dataframe(
      df_relates_to_N,
      source_id_space       = "Airport", 
      destination_id_space  = "Airport",   
      source_column         = "START_ID",
      destination_column    = "END_ID",
      type                  = "FLIGHT"
      )
         #
   df_importer.node_id_property_name("id")
      #
   df_importer.insert()
 
print("--")


          0/? [?op/s]

          0/? [?op/s]

--


#  Verifying Results ..

In [28]:

display(my_graph.num_nodes())
display(my_graph.num_edges())


          0/? [?op/s]

4

          0/? [?op/s]

4

In [19]:

#  What's in the graph ..

l_result = my_graph.query_unpaginated("""
   MATCH ( n ) 
   // RETURN COUNT(*)
   RETURN n
   """)

print(tabulate(l_result, headers='keys', tablefmt='psql'))
   #
print("--")


          0/? [?op/s]


--


In [33]:

#  What's in the graph ..

l_result = my_graph.query_unpaginated("""
   MATCH (n) - [r] -> (m)
   RETURN COUNT(r)
   // RETURN r
   """)
print(tabulate(l_result, headers='keys', tablefmt='psql'))


          0/? [?op/s]

+----+------------+
|    |   COUNT(r) |
|----+------------|
|  0 |          4 |
+----+------------+
